***
# __데이터 전처리__
***
# 1. 서울시 인구 전처리

### 1-1. 서울시 인구데이터 로드 및 전처리

In [31]:
import pandas as pd

In [32]:
popul_df = pd.read_csv('../Starbucks Data/자치구별 주민등록인구 (2023년 2분기).csv')
popul_df.head()

,동별(1),동별(2),2023 1/4,2023 1/4.1,2023 1/4.2,2023 1/4.3,2023 1/4.4,2023 1/4.5,2023 1/4.6,2023 1/4.7,...,2023 3/4.2,2023 3/4.3,2023 3/4.4,2023 3/4.5,2023 3/4.6,2023 3/4.7,2023 3/4.8,2023 3/4.9,2023 3/4.10,2023 3/4.11
0,동별(1),동별(2),세대 (세대),계 (명),계 (명),계 (명),한국인 (명),한국인 (명),한국인 (명),등록외국인 (명),...,계 (명),계 (명),한국인 (명),한국인 (명),한국인 (명),등록외국인 (명),등록외국인 (명),등록외국인 (명),인구밀도,인구밀도
1,동별(1),동별(2),소계,소계,남자,여자,소계,남자,여자,소계,...,남자,여자,소계,남자,여자,소계,남자,여자,소계,인구밀도 (명/㎢)
2,합계,소계,4463385,9668008,4672170,4995838,9426404,4566299,4860105,241604,...,4660459,4995459,9407540,4552501,4855039,248378,107958,140420,2,1725801
3,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,...,72222,78905,139945,67625,72320,11182,4597,6585,2,28610
4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,...,63661,68164,121405,58788,62617,10420,4873,5547,2,25737


In [33]:
# 데이터프레임 전처리 (지역구, 인구수, 인구밀도 컬럼만 추출 / 컬럼명 재설정)
main_df = popul_df.iloc[:, [1, 15, 25]]
main_df.columns = main_df.iloc[0]
main_df = main_df.drop(main_df.index[0:2])
main_df.rename(columns={'동별(2)':'자치구', '계 (명)':'인구 수 (명)', '인구밀도':'인구밀도 (명/㎢)'}, inplace=True)
main_df.drop(main_df.index[0], inplace=True)
main_df.index = list(range(1, len(main_df) + 1))
main_df.head()

,자치구,인구 수 (명),인구밀도 (명/㎢)
1,종로구,151349,28383
2,중구,132054,25546
3,용산구,232069,39717
4,성동구,286564,48514
5,광진구,350820,55418


In [34]:
# 지역구 이름순 정렬
main_df = main_df.sort_values(by='자치구')
main_df.index = list(range(1, len(main_df) + 1))
main_df.head()

,자치구,인구 수 (명),인구밀도 (명/㎢)
1,강남구,542327,84271
2,강동구,463005,80589
3,강북구,295773,67566
4,강서구,572382,99690
5,관악구,502430,83932


In [35]:
# 지역구 오름차순 리스트 추출
cities = list(main_df['자치구'])
cities[:6]

['강남구', '강동구', '강북구', '강서구', '관악구', '광진구']

In [36]:
# 자치구별 위도, 경도 추가
seoul_sgg_list = pd.read_excel('../Starbucks Data/자치구별 위도 경도.xlsx')
seoul_sgg_list.rename(columns={'시군구명':'자치구'}, inplace=True)
main_df = pd.merge(main_df, seoul_sgg_list, how='left', on='자치구')
main_df.drop('시군구코드', axis=1, inplace=True)
main_df.index = list(range(1, len(main_df) + 1))

main_df.head()

,자치구,인구 수 (명),인구밀도 (명/㎢),위도,경도
1,강남구,542327,84271,37.495985,127.066409
2,강동구,463005,80589,37.549208,127.146482
3,강북구,295773,67566,37.646995,127.014716
4,강서구,572382,99690,37.565762,126.822656
5,관악구,502430,83932,37.465399,126.943807


In [37]:
# 컬럼 순서 재배치
cols = ['자치구', '위도', '경도', '인구 수 (명)', '인구밀도 (명/㎢)']
main_df = main_df[cols]
main_df.head()

,자치구,위도,경도,인구 수 (명),인구밀도 (명/㎢)
1,강남구,37.495985,127.066409,542327,84271
2,강동구,37.549208,127.146482,463005,80589
3,강북구,37.646995,127.014716,295773,67566
4,강서구,37.565762,126.822656,572382,99690
5,관악구,37.465399,126.943807,502430,83932


In [38]:
# 데이터 타입 확인
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 1 to 25
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   자치구         25 non-null     object 
 1   위도          25 non-null     float64
 2   경도          25 non-null     float64
 3   인구 수 (명)    25 non-null     object 
 4   인구밀도 (명/㎢)  25 non-null     object 
dtypes: float64(2), object(3)
memory usage: 1.2+ KB


In [39]:
# 데이터 정수형으로 전환 (object -> itnt64)
main_df['인구 수 (명)'] = pd.to_numeric(main_df['인구 수 (명)'], errors='coerce')
main_df['인구밀도 (명/㎢)'] = pd.to_numeric(main_df['인구밀도 (명/㎢)'], errors='coerce')
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 1 to 25
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   자치구         25 non-null     object 
 1   위도          25 non-null     float64
 2   경도          25 non-null     float64
 3   인구 수 (명)    25 non-null     int64  
 4   인구밀도 (명/㎢)  25 non-null     int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 1.2+ KB


# 2. 자치구별 인구와 스타벅스 매칭

### 2-1. 자치구별 스타벅스 지점 수 도출

In [40]:
starbucks_df = pd.read_excel('../Starbucks Data/Starbucks_list.xlsx')
starbucks_df.head()

,매장명,매장 타입,자치구,매장 주소,위도,경도
0,역삼아레나빌딩점,pin_general,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리점,pin_general,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩점,pin_general,강남구,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606
3,국기원사거리점,pin_general,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩R점,pin_reserve,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583


In [41]:
# 자치구별 지점 개수 구하기
store_num = []
for city in cities:
    store_num.append(len(starbucks_df[starbucks_df['매장 주소'].str.contains(city)]))

store_num[:6]

[90, 17, 6, 27, 12, 18]

In [42]:
# 스타벅스 지점 수 컬럼 생성
main_df['스타벅스 지점 수'] = store_num
main_df.head()

,자치구,위도,경도,인구 수 (명),인구밀도 (명/㎢),스타벅스 지점 수
1,강남구,37.495985,127.066409,542327,84271,90
2,강동구,37.549208,127.146482,463005,80589,17
3,강북구,37.646995,127.014716,295773,67566,6
4,강서구,37.565762,126.822656,572382,99690,27
5,관악구,37.465399,126.943807,502430,83932,12


In [43]:
# 리저브 매장 개수컬럼 추가
reserve_df = starbucks_df.copy()
reserve_df.set_index('자치구', inplace=True)

starbucks_df.set_index('자치구')
reserve_mask = starbucks_df['매장 타입'] == 'pin_reserve'
reserve_df = starbucks_df[reserve_mask]

# 자치구별 스타벅스 매장 수 세기
reserve_count = reserve_df.pivot_table(
                                    index='자치구',
                                    values='매장 타입',
                                    aggfunc='count'
                                    ).rename(columns={'매장 타입':'리저브 매장'})

# 데이터프레임 병합 및 결측치 처리
main_df = pd.merge(main_df, reserve_count, how='left', on='자치구')
main_df['리저브 매장'].fillna(0, inplace=True)
main_df['리저브 매장'] = main_df['리저브 매장'].astype('int')
main_df.index = list(range(1, len(main_df) + 1))

main_df.head()

,자치구,위도,경도,인구 수 (명),인구밀도 (명/㎢),스타벅스 지점 수,리저브 매장
1,강남구,37.495985,127.066409,542327,84271,90,11
2,강동구,37.549208,127.146482,463005,80589,17,1
3,강북구,37.646995,127.014716,295773,67566,6,0
4,강서구,37.565762,126.822656,572382,99690,27,1
5,관악구,37.465399,126.943807,502430,83932,12,1


In [44]:
# 지점 당 인구수 생성
main_df['지점 당 인구수 (명)'] = main_df['인구 수 (명)'] / main_df['스타벅스 지점 수']
main_df['지점 당 인구수 (명)'] = main_df['지점 당 인구수 (명)'].astype(int)
main_df.head()

,자치구,위도,경도,인구 수 (명),인구밀도 (명/㎢),스타벅스 지점 수,리저브 매장,지점 당 인구수 (명)
1,강남구,37.495985,127.066409,542327,84271,90,11,6025
2,강동구,37.549208,127.146482,463005,80589,17,1,27235
3,강북구,37.646995,127.014716,295773,67566,6,0,49295
4,강서구,37.565762,126.822656,572382,99690,27,1,21199
5,관악구,37.465399,126.943807,502430,83932,12,1,41869


# 3. 자치구별 소득소비 지표 비교

### 3-1. 서울시 자치구 소득소비데이터 로드 및 전처리

In [45]:
inout_df = pd.read_csv('../Starbucks Data/자치구별 소득소비액 (2023년 2분기).csv', encoding='cp949')
inout_df.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,총식료품지출액,총의류신발지출액,총생활용품지출액,총의료비지출액,총교통지출액,총교육지출액,총유흥지출액,총여가문화지출액,총기타지출액,총음식지출액
0,20232,11500,강서구,3160623,7,196544942000,34606803000,6416265000,2093899000,31754728000,29615573000,6447544000,4985539000,25650372000,17705002000,37269217000
1,20232,11710,송파구,3995905,8,200981264000,40430085000,7840388000,4078692000,39623712000,9276542000,10273855000,5568234000,28050266000,8217527000,47621963000
2,20232,11200,성동구,3734499,7,87762952000,21437027000,4023393000,1383930000,13783407000,5776216000,2644736000,2789616000,5093632000,4323419000,26507576000
3,20232,11305,강북구,2547460,6,61629372000,15846136000,1782119000,1171002000,10614563000,3354150000,1603207000,2908287000,3851490000,2893375000,17605043000
4,20232,11470,양천구,3796034,8,106593132000,27174250000,1638040000,1352298000,20034954000,5658841000,12535282000,2204858000,11333675000,3926932000,20734002000


In [46]:
# 자치구별 소득소비 데이터프레임 전처리
inout_df = inout_df.iloc[:, [2, 3, 4, 5]]
inout_df.rename(columns={'행정동_코드_명':'자치구', '월_평균_소득_금액':'월평균소득', '소득_구간_코드':'소득구간',\
    '지출_총금액':'총지출액'}, inplace=True)
inout_df.sort_values(by='자치구', inplace=True)
inout_df.index = list(range(1, len(inout_df) + 1))
inout_df = inout_df[['자치구', '소득구간', '월평균소득', '총지출액']]
inout_df.head()

,자치구,소득구간,월평균소득,총지출액
1,강남구,9,4943841,2608201860000
2,강동구,7,3292857,160400134000
3,강북구,6,2547460,61629372000
4,강서구,7,3160623,196544942000
5,관악구,6,2771927,118122959000


In [47]:
# 자치구별 정보 데이터프레임 최종 병합 
final_df = pd.merge(main_df, inout_df, how='left', on='자치구')
final_df.index = list(range(1, len(final_df) + 1))
final_df.head()

,자치구,위도,경도,인구 수 (명),인구밀도 (명/㎢),스타벅스 지점 수,리저브 매장,지점 당 인구수 (명),소득구간,월평균소득,총지출액
1,강남구,37.495985,127.066409,542327,84271,90,11,6025,9,4943841,2608201860000
2,강동구,37.549208,127.146482,463005,80589,17,1,27235,7,3292857,160400134000
3,강북구,37.646995,127.014716,295773,67566,6,0,49295,6,2547460,61629372000
4,강서구,37.565762,126.822656,572382,99690,27,1,21199,7,3160623,196544942000
5,관악구,37.465399,126.943807,502430,83932,12,1,41869,6,2771927,118122959000


### 최종 데이터프레임 엑셀파일 저장

In [48]:
final_df.to_excel('../Starbucks Data/final_df.xlsx', index=False)

***
## 번외) 스타벅스 리스트 지역구별로 엑셀 시트 나누기

In [49]:
import pandas as pd
Starbucks_list_df = pd.read_excel('../Starbucks Data/Starbucks_list.xlsx', 'r')
Starbucks_list_df.head()

,매장명,매장 타입,자치구,매장 주소,위도,경도
0,역삼아레나빌딩점,pin_general,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리점,pin_general,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩점,pin_general,강남구,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606
3,국기원사거리점,pin_general,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩R점,pin_reserve,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583


In [50]:
writer = pd.ExcelWriter('../Starbucks Data/Starbucks_list.xlsx')
Starbucks_list_df.to_excel(writer, sheet_name='전국')

for city in cities:
    filt = Starbucks_list_df['자치구'].str.contains(city)
    city_df = Starbucks_list_df[filt]
    city_df.index = list(range(1, len(city_df) + 1))
    city_df.to_excel(writer, sheet_name=city)

writer.close()